In [1]:
import tensorflow as tf
import json
import matplotlib.pyplot as plt
import numpy as np

In [2]:
GPU = tf.config.list_physical_devices('GPU')
CPU = tf.config.list_physical_devices('CPU')

In [4]:
print(f'Available CPU/s:{len(CPU)}')
print(f'Available GPU/s:{len(GPU)}')

Available CPU/s:1
Available GPU/s:1


In [5]:
for gpu in GPU:
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [7]:
vgg = VGG16(include_top=False)

In [8]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [11]:
def build_model():
    input_layer = Input(shape=(120, 120, 3))

    vgg = VGG16(include_top=False)(input_layer)

    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)
    class2 = Dense(10)(class1)

    ASL_Identifier = Model(inputs=input_layer, outputs=[class2])

    return ASL_Identifier


In [12]:
ASL_Identifier = build_model()

In [13]:
ASL_Identifier.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 120, 120, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_max_pooling2d (Globa  (None, 512)              0         
 lMaxPooling2D)                                                  
                                                                 
 dense (Dense)               (None, 2048)              1050624   
                                                                 
 dense_1 (Dense)             (None, 10)                20490     
                                                                 
Total params: 15,785,802
Trainable params: 15,785,802
Non-trainable params: 0
_________________________________________________

In [15]:
ASL_Identifier.compile(optimizer='adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])